In [1]:
import sys, re
import datetime, time
import pandas as pd
from bs4 import BeautifulSoup
import pandas

In [44]:
SCHOOL = [('Columbia U', 'Columbia University'),
         ('Stanford','Stanford University'),
         ('Of Michi|Ann Arbor','University Of Michigan (Ann Arbor)'),
         ('UC B|Berkeley|California Berkeley|UCB','University Of California Berkeley'),
         ('Of Wash','University Of Washington'),
         ('Massachusetts Institute|MIT','Massachusetts Institute Of Technology (MIT)'),
         ('Cornell W|Cornell U|Weill C','Cornell University'),
         ('Wiscons','University Of Wisconsin'),
         ('Southern Cal|USC','University Of Southern California'),
         ('y Of Chi|Harris','University Of Chicago'),
         ('Toronto|Rotman|OISE|Oise|oise|UofT|UTSC|Scarborough','University Of Toronto'),
         ('Harvard','Harvard University'),
         ('Texas At|Austin|Of Texas','University Of Texas'),
         ('Carnegie|CMU','Carnegie Mellon University'),
         ('Northwest','Northwestern University'),
         ('Princeton','Princeton University'),
         ('Yale','Yale University'),
         ('Boston U|BU','Boston University'),
         ('Of Penn|UPenn','University Of Pennsylvania'),
         ('UCLA|Californa Los','University Of California Los Angeles (UCLA)'),
         ('Duke','Duke University'),
         ('New York U|NYU','New York Universit'),
         ('Oxford','University Of Oxford'),
         ('Ohio State|OSU','Ohio State University'),
         ('Minnesota','University Of Minnesota'),
         ('Johns Hopkins|JHU|Hopkins','Johns Hopkins University'),
         ('Georgia In|GT|GTech|Georgia T','Georgia Institute Of Technology'),
         ('Of Colo|Colorado B|Colorado D|Colorado Sp|CU B|CU D','University Of Colorado'),
         ('Of Massachusetts','University Of Massachusetts'),
         ('Of Maryland|UMD|UMCP|UMBC','University Of Maryland'),
         ('Of California I|UCI','University Of California Irvine'),
         ('TAMU|A&M','Texas A&M University - College Station (TAMU)'),
         ('Brown','Brown University'),
         ('f British C|UBCO|Sauder|Okanagan|UBC','University Of British Columbia'),
         (' San Diego|UCSD','University Of California San Diego'),
         ('Cambridge','University Of Cambridge'),
         ('Emory','Emory University'),
         ('California Davis|UCD','University Of California Davis'),
         ('McGill','McGill University'),
         ('f Virginia|UVA','University Of Virginia'),
         ('Carolina C|Chapel Hill|UNC','UNC Chapel Hill'),
         ('Northeastern|NEU','Northeastern University'),
         ('WashU|WUSTL|n St. Louis|WashU/WUSTL','Washington University In St. Louis (WashU/WUSTL)'),
         ('Arizona S|ASU','Arizona State University'),
         ('Bloomington|IU Bloomington','Indiana University Bloomington'),
         ('Purdue|West La','Purdue University'),
         ('Penn S|PSU|Pennsylvania St','Penn State University'),
         ('Vanderbilt','Vanderbilt University'),
         ('California Institute Of Te|Caltech|CalTech','California Institute Of Technology'),
         ('Of New York|CUNY','City University Of New York (CUNY)'),
         ('f Illinois|UIC|UIUC|Champaign','University Of Illinois'),
         ('Pittsburgh','University Of Pittsburgh'),
         ('Waterloo','University Of Waterloo')]

In [45]:
PROG = [('Computer Sci|ComputerSci', 'Computer Science'),
        ('Data Sci', 'Data Science'),
        ('Electrical And Computer Engineering|ECE|EECS', 'Electrical And Computer Engineering'),
        ('Arch', 'Architecture'),
        ('Material', 'Materials Science And Engineering'),
        ('Econ|Economics', 'Economics'),
        ('Physics', 'Physics'),
        ('Clinical Psychology', 'Clinical Psychology'),
        ('Speech Lang', 'Speech Language Pathology')]

In [326]:
def process(col):
    gpa_match = re.compile("GPA ((?:[0-9]\.[0-9]{1,2})|(?:n/a))")
    gre_match = re.compile("GRE 1(\d{2})")
    decdate_match = re.compile("\t")
    degree_match = re.compile("PhD|Masters|MFA|MBA|JD|EdD|IND|PsyD|Other")
    
    inst, major, decision, decmonth, gpa = None,None, None, None, None
    gre, status, entryTime, degree, comments= None, None, None,None,None
    
    lines = []
    for line in col:
        if line != '':
            if line != ' 0':
                lines.append(line)
    progtext = lines[0].strip().split('  ')
    if len(progtext) == 1:
        comments = ''
    else:
        comments = progtext[1].strip()
        
    other = lines[1:-1]
    for i, j in enumerate(other): 
        if j.startswith('-'):
            more_comments = other[i]
            more_comments = more_comments.replace('-', '')
            comments += more_comments
    
    majorSchool = progtext[0].strip()
    parts2 = majorSchool.split(', ')
    
    major_original = parts2[0].strip()
    for match,major_name in PROG:
        s = re.search(match, major_original)
        if s is not None:
            major = major_name
            break
        else:
            major = major_original
            
    inst_original =' '.join(parts2[1:])
    for match,inst_name in SCHOOL:
        s = re.search(match, inst_original)
        if s is not None:
            inst = inst_name
            break
        else:
            inst = inst_original
            

    for i, j in enumerate(lines):
        gpa = None
        if j.startswith('GPA 3') and j[-1].isdigit():
            if "/" not in j:
                gpa_list = lines[i]
                gpa = float(gpa_list.split()[-1])
                break

    gre_list =  ' '.join([str(elem) for elem in list(filter(gre_match.match, lines))])
    if gre_list == '':
        gre = None
    else: 
        gre = float(gre_list.split()[-1])    

    for i, j in enumerate(lines):
        if j == 'Accepted' or j == 'Accepted Other':
            decision = lines[i]
        elif j == 'Interview' or j == 'Interview Other':
            decision = lines[i]
        elif j == 'Rejected' or j == 'Rejected Other':
            decision = lines[i]
        elif j == 'Wait listed' or j == 'Wait listed Other':
            decision = lines[i]
        elif j == 'Other' or j == 'Other Other':
            decision = lines[i]

    for i, j in enumerate(lines):
        if j.startswith('\t\t'):
            decision_date = lines[i]
            decision_date = decision_date.replace('\t', '').replace(' on ', '')
    #decision_date= ' '.join([str(elem) for elem in list(filter(decdate_match.match, lines))])
    #decision_date = decision_date.replace('\t', '').replace(' on ', '')
            if decision_date == '29 Feb':
                decmonth = 'February'
            elif decision_date.startswith(' '):
                decision_date = decision_date.lstrip()
                decdate_date = datetime.datetime.strptime(decision_date, '%d %b')
                decmonth = decdate_date.strftime('%B')
            else:
                decdate_date = datetime.datetime.strptime(decision_date, '%d %b')
                decmonth = decdate_date.strftime('%B')


    for i, j in enumerate(lines):
        if j == 'International':
            status = lines[i]
        elif j == 'American':
            status = lines[i]
        elif j == 'Other':
            status = lines[i]

    for i, j in enumerate(lines):
        if len(j)==9:
            if j.startswith('F') or j.startswith('S'):
                entryTime = lines[i]

    for i, j in enumerate(lines):
        if j == 'PhD' or j == 'Masters' or j == 'MFA' or j == 'MBA' or j == 'JD' :
            degree = lines[i]
        elif j == 'EdD' or j == 'IND' or j == 'PsyD' or j == 'Other':
            degree = lines[i]

    results = [inst, major, decision, decmonth, gpa, gre, status, entryTime, degree, comments]
    return results


In [327]:
def parser(num_pages,school,major):
    DataSet = []  
    for idx in range(1,int(num_pages)): 
        with open('data/{0}.html'.format(idx),'r', encoding="utf8") as html_file:
            content = html_file.read()
            soup = BeautifulSoup(content,'lxml')
            tables = soup.findAll('div', class_='row mb-2')
            for tab in tables:
                rows = tab.text.splitlines()
                if len(rows)!=5:
                    values = process(rows)
                    DataSet.append(values)
            print('parsing page {}...'.format(idx))
    df = pd.DataFrame(DataSet,columns = ['school', 'major', 'decision', 'decision_month', 'gpa', 'gre', 'status', 'entryTime', 'degree', 'comments'])
    #df.to_csv('{}_raw.csv'.format(school+major),mode='w+')
    return df

In [328]:
df = parser(50,'','')

parsing page 1...
parsing page 2...
parsing page 3...
parsing page 4...
parsing page 5...
parsing page 6...
parsing page 7...
parsing page 8...
parsing page 9...
parsing page 10...
parsing page 11...
parsing page 12...
parsing page 13...
parsing page 14...
parsing page 15...
parsing page 16...
parsing page 17...
parsing page 18...
parsing page 19...
parsing page 20...
parsing page 21...
parsing page 22...
parsing page 23...
parsing page 24...
parsing page 25...
parsing page 26...
parsing page 27...
parsing page 28...
parsing page 29...
parsing page 30...
parsing page 31...
parsing page 32...
parsing page 33...
parsing page 34...
parsing page 35...
parsing page 36...
parsing page 37...
parsing page 38...
parsing page 39...
parsing page 40...
parsing page 41...
parsing page 42...
parsing page 43...
parsing page 44...
parsing page 45...
parsing page 46...
parsing page 47...
parsing page 48...
parsing page 49...


In [329]:
df['gpa'].unique()

array([ nan, 3.6 , 3.5 , 3.8 , 3.72, 3.98, 3.7 , 3.63, 3.76, 3.36, 3.69,
       3.55, 3.95, 3.96, 3.97, 3.77, 3.68, 3.94, 3.89, 3.9 , 3.71, 3.57,
       3.82, 3.93, 3.64, 3.65, 3.73, 3.84, 3.91, 3.88, 3.13, 3.99, 3.54,
       3.74, 3.92, 3.42, 3.35, 3.67, 3.85, 3.75, 3.3 , 3.38, 3.1 , 3.25,
       3.4 , 3.15, 3.2 , 3.83, 3.56, 3.86, 3.31, 3.23, 3.46, 3.  , 3.01,
       3.87, 3.58, 3.79, 3.53, 3.78, 3.51, 3.62, 3.44, 3.52, 3.48, 3.45,
       3.81, 3.66, 3.61, 3.43, 3.11, 3.17, 3.22, 3.33, 3.06, 3.39, 3.59])

In [320]:
df = df[df['school'].map(df['school'].value_counts()) > 2]

In [321]:
df['school'].value_counts()

University Of Chicago                94
University Of Washington             87
University Of California Berkeley    82
Stanford University                  71
Carnegie Mellon University           67
                                     ..
Baylor University                     3
Portland State University             3
Brandeis University                   3
Union Theological Seminary            3
University Of Miami                   3
Name: school, Length: 111, dtype: int64

In [313]:
major = ''

In [315]:
major_original = 'University Of California Berkeley - Haas School Of Business'
for match,major_name in PROG:
    s = re.search(match, major_original)
    if s is not None:
        major = major_name
        break
    else:
        major = major_original

In [316]:
major

'University Of California Berkeley'

In [298]:
df

,school,major,decision,decision_month,gpa,gre,status,entryTime,degree,comments
0,University Of California Berkeley,Earth And Planetary Science,Rejected,February,,,International,Fall 2022,PhD,I got an email from POI. I successfully contac...
1,University Of Chicago,Linguistics,Rejected,February,3.6,,International,Fall 2022,PhD,Email from a faculty. Referred my application ...
2,University Of Michigan (Ann Arbor),Materials Science And Engineering,Interview,None,,,American,Fall 2022,PhD,
3,University Of California Santa Cruz,Politics,Accepted,February,,,American,Fall 2022,PhD,"Informal phone call, awaiting formal letter"
4,Texas A&M University - College Station (TAMU),Public Health,Accepted,February,3.5,160,American,Fall 2022,Masters,
...,...,...,...,...,...,...,...,...,...,...
19955,University Of Wisconsin-Madison,Economics,Interview,None,,,None,Fall 2021,PhD,TBP:what does your last name start with? Anyon...
19956,CSU East Bay,Speech Language Pathology,Wait listed,None,,,None,Fall 2021,Masters,Ranked 6th-10th position. One of my top choice...
19957,Massachusetts Institute Of Technology (MIT),Economics,Wait listed,None,,,American,Fall 2021,PhD,Anyone else who was notified by MIT? What did ...
19958,Northwestern University,Economics,Wait listed,None,,,International,Fall 2021,PhD,Was waitlisted a while ago. Was anyone who was...
